

Quantum Neural Network for Methane Detection

This implementation follows a comprehensive approach combining quantum and classical techniques:

1. Data Preparation:
   - Loads temporal meteorological data
   - Handles missing values and outliers
   - Extracts temporal patterns and weather features
   - Normalizes data for quantum compatibility

2. Quantum Feature Encoding:
   - Uses angle encoding for quantum features
   - Maps features to rotation gates
   - Creates quantum-inspired classical features

3. Quantum Kernels:
   - Implements quantum kernel for similarity measures
   - Uses fidelity-based kernel computation
   - Handles quantum state preparation

4. QNN Architecture:
   - Combines quantum and classical components
   - Uses variational quantum circuits
   - Implements hybrid optimization

5. Evaluation:
   - Cross-validation for robustness
   - Multiple performance metrics
   - Cluster analysis for patterns

Author: Baby Vennela Kothakonda

Date: January 7, 2025

In [ ]:
# Install Neccessary Libraries
!pip install qiskit>=0.44.0
!pip install qiskit-aer>=0.12.0
!pip install qiskit-machine-learning>=0.6.0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

In [ ]:


class MethaneQNN:
    def __init__(self):
        """Initialize the Methane QNN with quantum and classical components"""
        # Initialize with None, will be set after PCA
        self.n_qubits = None
        self.qr = None
        self.cr = None

        # Initialize classical models with optimized parameters
        self.classical_models = {
            'random_forest': RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                n_jobs=-1,  # Use all CPU cores
                random_state=42
            ),
            'gradient_boost': GradientBoostingClassifier(
                n_estimators=100,
                max_depth=5,
                learning_rate=0.1,
                random_state=42
            ),
            'logistic': LogisticRegression(
                max_iter=1000,  # Increased iterations
                solver='lbfgs',
                n_jobs=-1,  # Use all CPU cores
                random_state=42
            )
        }

    def prepare_data(self, data):
        """
        Step 1: Data Preparation
        - Implements comprehensive data preprocessing
        - Extracts temporal, geographic, and weather features
        - Handles data cleaning and normalization
        """
        print("Processing features...")

        # 1.1 Temporal Feature Extraction
        # Convert time to datetime and extract cyclical features
        data['Time'] = pd.to_numeric(data['Time'])
        data['hour'] = data['Time'].astype(int)
        data['minute'] = ((data['Time'] % 1) * 60).astype(int)
        data['is_daytime'] = (data['hour'] >= 6) & (data['hour'] <= 18)

        # 1.2 Cyclical Time Encoding
        # Map time to circular coordinates to preserve periodicity
        data['hour_sin'] = np.sin(2 * np.pi * data['hour']/24)
        data['hour_cos'] = np.cos(2 * np.pi * data['hour']/24)

        # 1.3 Geographic Feature Engineering
        # Normalize spatial coordinates and calculate relative positions
        data['lat_norm'] = (data['latitude'] - data['latitude'].mean()) / data['latitude'].std()
        data['lon_norm'] = (data['longitude'] - data['longitude'].mean()) / data['longitude'].std()

        # Calculate distance from center
        center_lat = data['latitude'].mean()
        center_lon = data['longitude'].mean()
        data['dist_from_center'] = np.sqrt(
            (data['latitude'] - center_lat)**2 +
            (data['longitude'] - center_lon)**2
        )

        # 1.4 Weather Feature Engineering
        # Calculate derived weather parameters and stability indicators
        data['wind_speed'] = np.sqrt(
            data['u_west_to_east_wind']**2 +
            data['v_south_to_north_wind']**2
        )
        data['wind_direction'] = np.arctan2(
            data['v_south_to_north_wind'],
            data['u_west_to_east_wind']
        )

        # 1.5 Atmospheric Stability Features
        # Calculate gradients and stability indicators
        data['temp_gradient'] = data.groupby(['i_value', 'j_value'])['temprature'].diff()
        data['humidity_gradient'] = data.groupby(['i_value', 'j_value'])['relative_humidity'].diff()

        # 1.6 Physical Interaction Terms
        # Create physically meaningful feature interactions
        data['temp_humidity'] = data['temprature'] * data['relative_humidity']
        data['wind_temp'] = data['wind_speed'] * data['temprature']
        data['vapor_pressure'] = data['water_vapour'] * data['pressure']

        # 1.7 Feature Selection
        # Select features relevant for methane detection
        features = [
            # Temporal features for pattern detection
            'hour_sin', 'hour_cos', 'is_daytime',
            # Geographic features for spatial patterns
            'lat_norm', 'lon_norm', 'dist_from_center',
            # Weather features for atmospheric conditions
            'wind_speed', 'wind_direction',
            'temprature', 'relative_humidity',
            'vertical_velocity', 'pressure',
            'water_vapour', 'temp_gradient',
            'humidity_gradient', 'temp_humidity',
            'wind_temp', 'vapor_pressure',
            'turbulent_kinatic_energy',
            'precipitation_rate',
            'sensible_heat_flux',
            'Latent_heat_flux'
        ]

        # 1.8 Target Variable Creation
        print("Creating target variable...")
        data['target'] = (data['tracer_concentration'] > 0).astype(int)

        # 1.9 Class Balancing
        print("Balancing dataset...")
        pos_samples = data[data['target'] == 1]
        neg_samples = data[data['target'] == 0]

        # Undersample majority class
        n_samples = min(len(pos_samples), len(neg_samples))
        pos_balanced = pos_samples.sample(n=n_samples, random_state=42)
        neg_balanced = neg_samples.sample(n=n_samples, random_state=42)

        # Combine balanced datasets
        balanced_data = pd.concat([pos_balanced, neg_balanced])
        print(f"Class distribution after balancing - Positive: {(balanced_data['target'] == 1).mean():.2%}")

        # 1.10 Final Data Preparation
        X = balanced_data[features].copy()
        y = balanced_data['target'].values

        # Handle missing values
        X = X.fillna(X.mean())

        # 1.11 Feature Scaling
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X)

        # 1.12 Dimensionality Reduction
        print("Applying PCA...")
        self.pca = PCA(n_components=4)
        X_pca = self.pca.fit_transform(X_scaled)
        explained_var = np.sum(self.pca.explained_variance_ratio_)
        print(f"Explained variance with 4 components: {explained_var:.2%}")

        return X_pca, y

    def create_quantum_feature_map(self, n_features):
        """
        Step 2: Quantum Feature Encoding
        - Uses angle encoding for quantum features
        - Maps features to rotation gates
        """
        # Create ZZFeatureMap for angle encoding
        feature_map = ZZFeatureMap(
            feature_dimension=n_features,
            reps=2,
            entanglement='linear'
        )
        return feature_map

    def create_variational_circuit(self):
        """
        Step 3: Quantum Circuit Architecture
        - Combines quantum and classical components
        - Uses variational quantum circuits
        """
        # Create quantum circuit with focused architecture
        var_circuit = QuantumCircuit(self.qr, self.cr)

        # Single layer but with carefully chosen gates
        params_per_qubit = 2  # Rx and Ry rotations
        total_params = self.n_qubits * params_per_qubit
        params = ParameterVector('θ', length=total_params)

        # Apply rotations
        param_idx = 0
        for qubit in range(self.n_qubits):
            var_circuit.rx(params[param_idx], qubit)
            param_idx += 1
            var_circuit.ry(params[param_idx], qubit)
            param_idx += 1

        # Add entanglement - linear with periodic boundary
        for q in range(self.n_qubits):
            next_q = (q + 1) % self.n_qubits
            var_circuit.cx(q, next_q)

        # Add final rotations for measurement basis
        for qubit in range(self.n_qubits):
            var_circuit.h(qubit)

        # Add measurements
        var_circuit.measure(self.qr, self.cr)

        return var_circuit

    def train_quantum_kernel_model(self, X_train, y_train):
        """
        Step 4: Quantum Kernel Training
        - Implements quantum kernel for similarity measures
        - Uses fidelity-based kernel computation
        """
        print("\nTraining quantum kernel models...")

        # Create quantum kernel
        feature_map = self.create_quantum_feature_map(X_train.shape[1])
        quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

        # Generate kernel matrix
        print("Computing quantum kernel matrix...")
        try:
            kernel_matrix_train = quantum_kernel.evaluate(X_train)
        except Exception as e:
            print(f"Error computing kernel matrix: {str(e)}")
            print("Falling back to classical kernel...")
            from sklearn.metrics.pairwise import rbf_kernel
            kernel_matrix_train = rbf_kernel(X_train)

        # Initialize and train classical models
        print("\nTraining classical models with quantum kernel...")
        from sklearn.svm import SVC
        from sklearn.ensemble import RandomForestClassifier

        self.classical_models = {}

        # Train SVM with quantum kernel
        print("Training SVM...")
        svm = SVC(kernel='precomputed', probability=True, random_state=42)
        svm.fit(kernel_matrix_train, y_train)
        self.classical_models['svm'] = {'model': svm, 'kernel': quantum_kernel}

        # Train RF on original features
        print("Training RF...")
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        self.classical_models['rf'] = {'model': rf, 'kernel': None}

        print("Model training completed")

    def evaluate_quantum_kernel(self, X_test, y_test):
        """
        Step 5: Quantum Kernel Evaluation
        - Evaluates models with quantum kernel
        """
        print("\nEvaluating models...")

        results = {}
        for name, model_dict in self.classical_models.items():
            model = model_dict['model']
            kernel = model_dict['kernel']

            # Get predictions
            if kernel is not None:
                try:
                    # For SVM, compute kernel between test and training data
                    kernel_matrix_test = kernel.evaluate(X_test, self.X_train)
                    y_pred = model.predict(kernel_matrix_test)
                except Exception as e:
                    print(f"Error in kernel prediction: {str(e)}")
                    print("Using RBF kernel fallback...")
                    kernel_matrix_test = rbf_kernel(X_test, self.X_train)
                    y_pred = model.predict(kernel_matrix_test)
            else:
                # For RF, use features directly
                y_pred = model.predict(X_test)

            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

            results[name] = {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1_score': f1
            }

            print(f"\n{name.upper()} Results:")
            for metric, value in results[name].items():
                print(f"{metric}: {value:.4f}")

        return results

    def create_quantum_inspired_features(self, X):
        """
        Step 6: Quantum-Inspired Feature Generation
        - Creates quantum-inspired classical features
        - Implements angle encoding principles
        - Generates non-linear transformations
        """
        print("Creating quantum-inspired features...")

        # 6.1 Quantum Range Encoding
        X_quantum = (X + 4) * np.pi / 4  # Map to [0, 2π] range

        # 6.2 Quantum-Inspired Transformations
        features = []
        feature_names = []

        # 6.3 Original Features (like |0⟩ state)
        features.append(X_quantum)
        feature_names.extend([f'quantum_f{i}' for i in range(X_quantum.shape[1])])

        # 6.4 Rotation-Inspired Features (like Rx, Ry gates)
        features.append(np.sin(X_quantum))  # Rx-like
        features.append(np.cos(X_quantum))  # Ry-like
        feature_names.extend([f'sin_f{i}' for i in range(X_quantum.shape[1])])
        feature_names.extend([f'cos_f{i}' for i in range(X_quantum.shape[1])])

        # 6.5 Entanglement-Inspired Features (like CNOT effects)
        for i in range(X_quantum.shape[1]):
            for j in range(i+1, X_quantum.shape[1]):
                interaction = X_quantum[:, i] * X_quantum[:, j]
                features.append(interaction.reshape(-1, 1))
                feature_names.append(f'interact_f{i}_f{j}')

        # 6.6 Combine Features
        X_enhanced = np.hstack(features)
        self.feature_names = feature_names

        return X_enhanced

    def tune_and_train_models(self, X_train, y_train):
        """Tune hyperparameters and train models"""
        print("\nTuning and training enhanced models...")

        # Create quantum-inspired features
        print("Creating quantum-inspired features...")
        X_enhanced = self.create_quantum_inspired_features(X_train)

        # Simplified parameter grids for faster tuning
        param_grid = {
            'random_forest': {
                'n_estimators': [50, 100],
                'max_depth': [5, 10]
            },
            'gradient_boost': {
                'n_estimators': [50, 100],
                'max_depth': [3, 5]
            },
            'logistic': {
                'C': [0.1, 1.0]
            }
        }

        # Train each model with simplified tuning
        for name, model in self.classical_models.items():
            print(f"\nTuning {name}...")

            # Use 3-fold CV instead of 5-fold for speed
            grid_search = GridSearchCV(
                model,
                param_grid[name],
                cv=3,
                n_jobs=-1,  # Use all CPU cores
                scoring='f1'
            )

            grid_search.fit(X_enhanced, y_train)
            self.classical_models[name] = grid_search.best_estimator_

            print(f"Best parameters: {grid_search.best_params_}")
            print(f"Best CV score: {grid_search.best_score_:.4f}")

        print("\nModel training completed")

    def analyze_feature_importance(self):
        """
        Step 8: Feature Importance Analysis
        - Analyzes feature importance for classical models
        - Visualizes feature importance
        """
        print("\nAnalyzing feature importance...")

        import matplotlib.pyplot as plt

        for name, model in self.classical_models.items():
            if hasattr(model, 'feature_importances_'):
                # Get feature importances
                importances = model.feature_importances_
                indices = np.argsort(importances)[::-1]

                # Print feature ranking
                print(f"\nFeature ranking for {name}:")
                for f in range(len(self.feature_names)):
                    print(f"{f+1}. {self.feature_names[indices[f]]}: {importances[indices[f]]:.4f}")

                # Plot feature importances
                plt.figure(figsize=(10, 6))
                plt.title(f"Feature Importances ({name})")
                plt.bar(range(len(indices[:10])), importances[indices[:10]])
                plt.xticks(range(len(indices[:10])), [self.feature_names[i] for i in indices[:10]], rotation=45)
                plt.tight_layout()
                plt.savefig(f'feature_importance_{name}.png')
                plt.close()

                print(f"Feature importance plot saved as 'feature_importance_{name}.png'")

            elif hasattr(model, 'coef_'):
                # For logistic regression
                coef = np.abs(model.coef_[0])
                indices = np.argsort(coef)[::-1]

                print(f"\nFeature coefficients for {name}:")
                for f in range(len(self.feature_names)):
                    print(f"{f+1}. {self.feature_names[indices[f]]}: {coef[indices[f]]:.4f}")

                plt.figure(figsize=(10, 6))
                plt.title(f"Feature Coefficients ({name})")
                plt.bar(range(len(indices[:10])), coef[indices[:10]])
                plt.xticks(range(len(indices[:10])), [self.feature_names[i] for i in indices[:10]], rotation=45)
                plt.tight_layout()
                plt.savefig(f'feature_importance_{name}.png')
                plt.close()

                print(f"Feature coefficient plot saved as 'feature_importance_{name}.png'")

    def train_enhanced_model(self, X_train, y_train):
        """
        Step 9: Enhanced Model Training
        - Trains models with quantum-inspired features
        """
        print("\nTraining enhanced models...")

        # Create quantum-inspired features
        X_enhanced = self.create_quantum_inspired_features(X_train)

        # Initialize models
        from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
        from sklearn.linear_model import LogisticRegression

        models = {
            'rf': RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42),
            'gb': GradientBoostingClassifier(n_estimators=200, max_depth=5, random_state=42),
            'lr': LogisticRegression(max_iter=1000, random_state=42)
        }

        # Train models
        self.classical_models = {}
        for name, model in models.items():
            print(f"Training {name}...")
            model.fit(X_enhanced, y_train)
            self.classical_models[name] = model

        print("Model training completed")

    def evaluate_enhanced(self, X_test, y_test):
        """
        Step 10: Enhanced Model Evaluation
        - Evaluates models with quantum-inspired features
        """
        print("\nEvaluating models...")

        # Create quantum-inspired features for test set
        X_enhanced = self.create_quantum_inspired_features(X_test)

        results = {}
        predictions = {}

        for name, model in self.classical_models.items():
            # Get predictions
            y_pred = model.predict(X_enhanced)
            predictions[name] = y_pred

            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

            results[name] = {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1_score': f1
            }

            print(f"\n{name.upper()} Results:")
            for metric, value in results[name].items():
                print(f"{metric}: {value:.4f}")

        # Ensemble prediction (majority voting)
        ensemble_pred = np.mean([predictions[name] for name in predictions], axis=0) > 0.5

        # Calculate ensemble metrics
        accuracy = accuracy_score(y_test, ensemble_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, ensemble_pred, average='binary')

        results['ensemble'] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }

        print("\nENSEMBLE Results:")
        for metric, value in results['ensemble'].items():
            print(f"{metric}: {value:.4f}")

        return results

    def perform_clustering(self, X, n_clusters=3):
        """
        Step 11: Clustering Analysis
        - Performs classical clustering on the enhanced features
        """
        print("\nPerforming clustering...")

        # Create enhanced features for clustering
        X_enhanced = self.create_quantum_inspired_features(X)

        # Use MiniBatchKMeans for faster clustering
        from sklearn.cluster import MiniBatchKMeans

        print("Clustering data...")
        kmeans = MiniBatchKMeans(
            n_clusters=n_clusters,
            batch_size=1000,
            random_state=42
        )
        clusters = kmeans.fit_predict(X_enhanced)

        return clusters

    def visualize_clusters(self, X, clusters):
        """
        Step 12: Cluster Visualization
        - Visualizes clusters using PCA
        """
        print("Creating cluster visualization...")

        # Use PCA for visualization
        from sklearn.decomposition import PCA

        # Reduce to 2D for visualization
        pca = PCA(n_components=2)
        X_2d = pca.fit_transform(X)

        # Plot clusters
        import matplotlib.pyplot as plt

        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=clusters, cmap='viridis')
        plt.colorbar(scatter)
        plt.title('Cluster Visualization (PCA)')
        plt.xlabel('First Principal Component')
        plt.ylabel('Second Principal Component')
        plt.tight_layout()
        plt.savefig('quantum_clusters.png')
        plt.close()

        print("Cluster visualization saved as 'quantum_clusters.png'")

        # Print cluster statistics
        unique, counts = np.unique(clusters, return_counts=True)
        print("\nCluster sizes:")
        for cluster_id, count in zip(unique, counts):
            print(f"Cluster {cluster_id}: {count} samples ({count/len(clusters):.1%})")

    def cross_validate_models(self, X, y, n_splits=5):
        """
        Step 13: Cross-Validation
        - Performs cross-validation for robustness
        """
        print("\nPerforming cross-validation...")

        from sklearn.model_selection import KFold
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

        # Store results for each fold
        cv_results = {model_name: [] for model_name in self.classical_models.keys()}
        cv_results['ensemble'] = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
            print(f"\nFold {fold + 1}/{n_splits}")

            # Split data
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            # Create features
            X_train_enhanced = self.create_quantum_inspired_features(X_train)
            X_val_enhanced = self.create_quantum_inspired_features(X_val)

            # Train and evaluate each model
            fold_predictions = []

            for name, model in self.classical_models.items():
                # Train model
                model.fit(X_train_enhanced, y_train)

                # Get predictions
                y_pred = model.predict(X_val_enhanced)
                fold_predictions.append(y_pred)

                # Calculate metrics
                accuracy = accuracy_score(y_val, y_pred)
                precision = precision_score(y_val, y_pred)
                recall = recall_score(y_val, y_pred)
                f1 = f1_score(y_val, y_pred)

                cv_results[name].append({
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1
                })

                print(f"{name} - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

            # Ensemble predictions
            ensemble_pred = np.mean(fold_predictions, axis=0) > 0.5

            # Calculate ensemble metrics
            accuracy = accuracy_score(y_val, ensemble_pred)
            precision = precision_score(y_val, ensemble_pred)
            recall = recall_score(y_val, ensemble_pred)
            f1 = f1_score(y_val, ensemble_pred)

            cv_results['ensemble'].append({
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1
            })

            print(f"Ensemble - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

        # Print average results
        print("\nAverage Cross-Validation Results:")
        for name in cv_results.keys():
            avg_accuracy = np.mean([x['accuracy'] for x in cv_results[name]])
            avg_f1 = np.mean([x['f1'] for x in cv_results[name]])
            print(f"{name} - Avg Accuracy: {avg_accuracy:.4f}, Avg F1: {avg_f1:.4f}")

        return cv_results



In [ ]:
# Load and prepare data
print("Loading dataset...")
data = pd.read_csv('methane datain.csv')
print(f"Dataset shape: {data.shape}")

# Initialize QNN
qnn = MethaneQNN()

# Prepare data with enhanced features
print("Preparing data...")
X_scaled, y = qnn.prepare_data(data)
print(f"Prepared data shapes - X: {X_scaled.shape}, y: {y.shape}")

# Set number of qubits based on PCA components
qnn.n_qubits = X_scaled.shape[1]
qnn.qr = QuantumRegister(qnn.n_qubits)
qnn.cr = ClassicalRegister(qnn.n_qubits)

# Perform cross-validation
cv_results = qnn.cross_validate_models(X_scaled, y)

# Train final model on full dataset
print("\nTraining final models...")
qnn.tune_and_train_models(X_scaled, y)

# Optional: Analyze feature importance and clustering
# Comment these out if not needed
qnn.analyze_feature_importance()
clusters = qnn.perform_clustering(X_scaled)
qnn.visualize_clusters(X_scaled, clusters)


Loading dataset...
Dataset shape: (301340, 17)
Preparing data...
Processing features...
Creating target variable...
Balancing dataset...
Class distribution after balancing - Positive: 50.00%
Applying PCA...
Explained variance with 4 components: 67.01%
Prepared data shapes - X: (21812, 4), y: (21812,)

Performing cross-validation...

Fold 1/5
Creating quantum-inspired features...
Creating quantum-inspired features...
random_forest - Accuracy: 0.7891, F1: 0.8092
gradient_boost - Accuracy: 0.7848, F1: 0.8033
logistic - Accuracy: 0.7243, F1: 0.7487
Ensemble - Accuracy: 0.7825, F1: 0.8032

Fold 2/5
Creating quantum-inspired features...
Creating quantum-inspired features...
random_forest - Accuracy: 0.7956, F1: 0.8175
gradient_boost - Accuracy: 0.7956, F1: 0.8158
logistic - Accuracy: 0.7348, F1: 0.7602
Ensemble - Accuracy: 0.7901, F1: 0.8125

Fold 3/5
Creating quantum-inspired features...
Creating quantum-inspired features...
random_forest - Accuracy: 0.7861, F1: 0.8058
gradient_boost - Accu